# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## <span style='color:#2656a3'> 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Predict using model from Model Registry.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

First, we'll install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we'll proceed to import all the necessary libraries.

In [29]:
# Importing the packages for the needed libraries for the Jupyter notebook
import joblib
import inspect 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os

#%config InlineBackend.figure_format='retina'
#%matplotlib inline

## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

In [2]:
# Importing the hopsworks module
import hopsworks

# Logging in to the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store() 

c:\Users\Benj3\anaconda3\envs\energy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/556180
Connected. Call `.close()` to terminate connection gracefully.


### <span style='color:#2656a3'> ⚙️ Feature View Retrieval

In [3]:
# Retrieve the 'electricity_feature_view' feature view
feature_view = fs.get_feature_view(
    name='electricity_feature_view',
    version=1,
)

### <span style='color:#2656a3'> 🗄 Model Registry

In [4]:
# Retrieve the model registry
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


## <span style='color:#2656a3'> 📮 Retrieving model from Model Registry

In [6]:
# Retrieving the TensorFlow model from the Model Registry
model = mr.get_model(
    name="electricity_price_prediction_model", 
    version=1,
)

# Downloading the model to a local directory
model_dir = model.download()

# Loading the saved XGB model
model = joblib.load(model_dir + "/dk_electricity_model.pkl")


In [15]:
# display model
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style='color:#2656a3'> ✨ Load Batch Data

In [7]:
import datetime

# Calculating the start date as 5 days ago from the current date
start_date = datetime.datetime.now() - datetime.timedelta(days=5)

# Converting the start date to a timestamp in milliseconds
start_time = int(start_date.timestamp()) * 1000

In [8]:
# Initializing batch scoring
feature_view.init_batch_scoring(1)

# Retrieving batch data from the feature view starting from the specified start time
df = feature_view.get_batch_data(
    start_time=start_time,
)

Finished: Reading data from Hopsworks, using ArrowFlight (2.38s) 


In [11]:
# Sorting the DataFrame based on the 'timestamp' column
df.sort_values(["timestamp"], inplace=True)

# Dropping the 'day' and 'timestamp' columns from the DataFrame
df = df.drop(["date", "timestamp","time"], axis=1)

# Displaying the first 3 rows of the modified DataFrame
df.head(3)

,dk1_spotpricedkk_kwh,dk1_offshore_wind_forecastintraday_kwh,dk1_onshore_wind_forecastintraday_kwh,dk1_solar_forecastintraday_kwh,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m,type
26,0.220916,0.419822,0.138226,0.408640,0.425000,0.640000,0.011364,0.011364,0.0,0.68,1.0,0.234343,0.275681,1
1,0.218521,0.392039,0.132435,0.459908,0.443182,0.546667,0.000000,0.000000,0.0,0.04,1.0,0.204040,0.248428,1
6,0.212760,0.341092,0.106260,0.504301,0.447727,0.493333,0.000000,0.000000,0.0,0.04,1.0,0.173737,0.222222,1


## <span style='color:#2656a3'> 🤖 Testing Making the predictions

In [23]:
# Making predictions on the batch data using the retrieved XGBoost model
X_pred = df.iloc[:, 1:]

print("Daily instance: \n{}".format(X_pred))

Daily instance: 
     dk1_offshore_wind_forecastintraday_kwh  \
26                                 0.419822   
1                                  0.392039   
6                                  0.341092   
21                                 0.296045   
16                                 0.270865   
..                                      ...   
78                                 0.304475   
0                                  0.308764   
106                                0.348825   
74                                 0.433383   
50                                 0.517575   

     dk1_onshore_wind_forecastintraday_kwh  dk1_solar_forecastintraday_kwh  \
26                                0.138226                    4.086395e-01   
1                                 0.132435                    4.599078e-01   
6                                 0.106260                    5.043012e-01   
21                                0.088750                    5.030963e-01   
16                          

In [27]:
# predict and get latest (daily) feature
y_pred = model.predict(df.iloc[:, 0])
print("Prediction: {}".format(y_pred[0]))

ValueError: feature_names mismatch: ['dk1_offshore_wind_forecastintraday_kwh', 'dk1_onshore_wind_forecastintraday_kwh', 'dk1_solar_forecastintraday_kwh', 'temperature_2m', 'relative_humidity_2m', 'precipitation', 'rain', 'snowfall', 'weather_code', 'cloud_cover', 'wind_speed_10m', 'wind_gusts_10m', 'type'] ['dk1_spotpricedkk_kwh']
expected relative_humidity_2m, weather_code, dk1_solar_forecastintraday_kwh, type, dk1_offshore_wind_forecastintraday_kwh, precipitation, snowfall, wind_speed_10m, temperature_2m, dk1_onshore_wind_forecastintraday_kwh, cloud_cover, rain, wind_gusts_10m in input data
training data did not have the following fields: dk1_spotpricedkk_kwh

## <span style='color:#2656a3'> 🤖 Making the predictions

In [ ]:
# Using the serving function to make predictions for the given DataFrame
x = serving_function(
    tf.constant(
        df.values.reshape(-1, df.shape[0], df.shape[1]), tf.float32)
)

# Extracting and converting the predictions to a NumPy array
prediction = x[list(x.keys())[0]].numpy() #.flatten().tolist()
prediction

In [ ]:
# Extracting and organizing the predicted prices for different regions from the batch predictions
price_dk1_pred = []

for batch_n in range(len(prediction)):
    batch = prediction[batch_n]
    for window_n in range(df.shape[0]):
        price_dk1_pred.append(batch[window_n][0])

In [ ]:
# Initializing serving
feature_view.init_serving(1)

# Accessing the transformation functions used in the serving configuration
td_transformation_functions = feature_view._batch_scoring_server._transformation_functions

In [ ]:
# Decoding the features from the predicted prices
preds = pd.DataFrame(data={'price_dk1': price_dk1_pred,})

# Extracting and decoding the transformation functions used in serving
res = {}
for feature_name in td_transformation_functions:
    if feature_name in ["price_dk1"]:
        td_transformation_function = td_transformation_functions[feature_name]
        sig, foobar_locals = inspect.signature(td_transformation_function.transformation_fn), locals()
        param_dict = dict([(param.name, param.default) for param in sig.parameters.values() if param.default != inspect._empty])
        if td_transformation_function.name == "min_max_scaler":
            preds[feature_name] = preds[feature_name].map(
                lambda x: x * (param_dict["max_value"] - param_dict["min_value"]) + param_dict["min_value"]
                )

# Applying a transformation to reverse the sign of the decoded features
preds = preds.apply(lambda x: -x)
preds

In [ ]:
# Plotting the predicted electricity prices for different regions
plt.plot(preds["price_dk1"],color='red', label='DK1 price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc='upper left')

# Adjusting the size of the plot
fig = plt.gcf()
fig.set_size_inches(15, 5)

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Electricity price tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai